In [1]:
import os
import shutil
import csv
import pandas as pd
import numpy as np
from PIL import Image

In [2]:
# The FER labels already align with the universal emotion labels
# universal_emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
FER_Label_Mapping = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

In [3]:
def save_images(dataset, base_dir, split_name, target_dir):
    """
    Save images and create labels CSV for a dataset
    
    Parameters:
    dataset (DataFrame): The dataset containing image data
    split_name (str): Prefix for filenames (train/val/test)
    target_dir (str): Subdirectory to save images and CSV
    """
    label_data = []
    
    for idx, (_, row) in enumerate(dataset.iterrows()):
        # Process pixels
        pixels = np.array(list(map(int, row['pixels'].split())), dtype=np.uint8)
        img_array = pixels.reshape(48, 48)
        img = Image.fromarray(img_array)
        
        # Create filename components
        emotion_label = row['emotion']
        filename = f"{split_name}_{idx}_{FER_Label_Mapping[emotion_label]}.png"
        
        # Save image
        img.save(os.path.join(base_dir, target_dir, filename))
        
        # Store label information
        label_data.append({'filename': filename, 'label': emotion_label})
    
    # Create and save labels DataFrame
    labels_df = pd.DataFrame(label_data)
    labels_df.to_csv(os.path.join(base_dir, target_dir, 'labels.csv'), index=False)

In [4]:
def copy_images_to_label_subdirectories(base_dir):
    """
    For each subdirectory (test, train, validation) in base_dir,
    copy images into subdirectories based on their label, extracted from the filename.
    
    Expected filename format: {usecase}_{index}_{label}.{ext}
    """
    # Define the usage folders.
    usage_dirs = ['train', 'test', 'validation']
    
    for usage in usage_dirs:
        usage_path = os.path.join(base_dir, usage)
        if not os.path.isdir(usage_path):
            print(f"Directory {usage_path} does not exist. Skipping.")
            continue
        
        # Process each file in the usage folder.
        for filename in os.listdir(usage_path):
            file_path = os.path.join(usage_path, filename)
            if os.path.isfile(file_path):
                # Parse the filename. We expect at least 3 parts separated by '_'
                parts = filename.split('_')
                if len(parts) < 3:
                    print(f"Filename {filename} does not match expected format. Skipping.")
                    continue
                
                # The label is assumed to be the last part, with the file extension removed.
                label_with_ext = parts[-1]
                label, _ = os.path.splitext(label_with_ext)
                
                # Create the label subdirectory if it doesn't exist.
                label_dir = os.path.join(usage_path, label)
                os.makedirs(label_dir, exist_ok=True)
                
                # Copy the image into the label subdirectory.
                destination_file_path = os.path.join(label_dir, filename)
                shutil.copy2(file_path, destination_file_path)

In [5]:
# Read the CSV file
df = pd.read_csv('FER/fer2013.csv')

# Define base directory structure
base_directory = 'FER_Structured'
os.makedirs(base_directory, exist_ok=True)

# Create subdirectories
for folder in ['train', 'validation', 'test']:
    os.makedirs(os.path.join(base_directory, folder), exist_ok=True)

# Filter the dataset into three sets based on the value in 'Usage'
training_set = df[df['Usage'] == 'Training']
validation_set = df[df['Usage'] == 'PublicTest']
testing_set = df[df['Usage'] == 'PrivateTest']

# Save all sets with images and labels
save_images(training_set, base_directory, "train", "train")
save_images(validation_set, base_directory, "val", "validation")
save_images(testing_set, base_directory, "test", "test")

copy_images_to_label_subdirectories(base_directory)

Filename labels.csv does not match expected format. Skipping.
Filename labels.csv does not match expected format. Skipping.
Filename labels.csv does not match expected format. Skipping.


# Loading the FER2013 dataset

Note: Mappings are converting to be common with all other datasets

In [ ]:
# import pandas as pd

# # Read the CSV file
# df = pd.read_csv('FER/fer2013.csv')

# # Filter the dataset into three sets based on the value in 'Usage'
# training_set = df[df['Usage'] == 'Training']
# validation_set = df[df['Usage'] == 'PublicTest']
# testing_set = df[df['Usage'] == 'PrivateTest']

# # This will be the emotion mapping used throughout the project (for all datasets)
# universal_emotion_mapping = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Sad': 4, 'Surprise': 5, 'Neutral': 6}

# # Dataset specific emotion mapping (The mappings will be converted to the universal mapping for consistency)
# emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# # Display the first few rows of each set
# print("Training Set:")
# # Correctly mapping the labels to the universal emotion mapping
# training_set.loc[:, 'emotion'] = training_set['emotion'].map(emotion_mapping).map(universal_emotion_mapping)
# print(training_set.head())

# print("\nTesting Set:")
# # Correctly mapping the labels to the universal emotion mapping
# testing_set.loc[:, 'emotion'] = testing_set['emotion'].map(emotion_mapping).map(universal_emotion_mapping)
# print(testing_set.head())

# print("\nValidation Set:")
# # Correctly mapping the labels to the universal emotion mapping
# validation_set.loc[:, 'emotion'] = validation_set['emotion'].map(emotion_mapping).map(universal_emotion_mapping)
# print(validation_set.head())

Training Set:
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training

Testing Set:
       emotion                                             pixels       Usage
28709        0  254 254 254 254 254 249 255 160 2 58 53 70 77 ...  PublicTest
28710        1  156 184 198 202 204 207 210 212 213 214 215 21...  PublicTest
28711        4  69 118 61 60 96 121 103 87 103 88 70 90 115 12...  PublicTest
28712        6  205 203 236 157 83 158 120 116 94 86 155 180 2...  PublicTest
28713        3  87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...  PublicTest

Validation Set:
       emotion                                         

# Storing the images according to their use case 

Note: Images are named as follows {test/train/validation}\_{index}\_{emotion_label}

In [ ]:
# import os
# from PIL import Image
# import matplotlib.pyplot as plt
# import numpy as np

# destination_directory = 'FER_ResEmoteNet'
# universal_emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# # Create directories if they don't exist
# os.makedirs(destination_directory+'/train', exist_ok=True)
# os.makedirs(destination_directory+'/test', exist_ok=True)
# os.makedirs(destination_directory+'/validation', exist_ok=True)

# def save_images(dataset, set_name):
#     for index, row in dataset.iterrows():
#         emotion_label = universal_emotion_mapping[row['emotion']]
#         image_data = row['pixels']
#         image_pixels = np.fromstring(image_data, sep=' ').astype(int)
#         image_pixels = image_pixels.reshape(48, 48)
        
#         # Create an image from the pixel data
#         image = Image.fromarray(image_pixels.astype('uint8'), 'L')
        
#         # Save the image to the appropriate directory
#         image.save(f'{destination_directory}/{set_name}/{set_name}_{index}_{emotion_label.lower()}.jpg')

# # Save images for each set
# save_images(training_set, 'train')
# save_images(testing_set, 'test')
# save_images(validation_set, 'validation')

# Augmenting the data in the Training set

In [ ]:
# import os
# from PIL import Image, ImageOps
# import random

# dataset_path = os.path.join(destination_directory, 'train')
# target_num_images = 1000

# def augment_image(image):
#     """Apply random augmentations to an image."""
#     augmentations = [
#         lambda x: x.rotate(random.randint(-30, 30)),
#         lambda x: ImageOps.mirror(x),
#         lambda x: ImageOps.crop(x, border=random.randint(0, 10)),
#     ]
#     augmentation = random.choice(augmentations)
#     return augmentation(image)

# # Group images by emotion
# images_by_emotion = {}
# for filename in os.listdir(dataset_path):
#     if filename.endswith(('.jpg', '.png')):
#         emotion = filename.split('_')[-1].split('.')[0]
#         # print(emotion)
#         if emotion not in images_by_emotion:
#             images_by_emotion[emotion] = []
#         images_by_emotion[emotion].append(filename)

# # Augment images for each emotion
# for emotion, images in images_by_emotion.items():
#     num_images = len(images)
    
#     if num_images < target_num_images:
#         for i in range(target_num_images - num_images):
#             original_image_path = os.path.join(dataset_path, images[i % num_images])
#             with Image.open(original_image_path) as img:
#                 augmented_image = augment_image(img)
#                 new_image_name = f'train_augmented_{num_images + i + 1}_{emotion}.jpg'
#                 augmented_image.save(os.path.join(dataset_path, new_image_name))

# print("Image augmentation completed.")

Image augmentation completed.


In [ ]:
# file_extension = 'jpg'

# universal_emotion_mapping = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Sad': 4, 'Surprise': 5, 'Neutral': 6}

# folder_mapping = {'test': 'test', 'train': 'train', 'validation': 'val'}

# # Iterate over the folders (test, train, validation)
# for folder in ['test', 'train', 'validation']:
#     folder_path = os.path.join(destination_directory, folder)

#     image_data = []

#     # Make sure the name of the file is partition_iteration_emotion.jpg or .png
#     for filename in os.listdir(folder_path):
#         if filename.endswith(file_extension):  
#             label_name = filename.split('_')[-1].split('.')[0]
#             label_value = universal_emotion_mapping.get(label_name.capitalize())
#             if label_value is not None:  
#                 image_data.append([filename, label_value])

#     df = pd.DataFrame(image_data, columns=["ImageName", "Label"])

#     csv_file_path = os.path.join(folder_path, 'labels.csv')

#     df.to_csv(csv_file_path, index=False, header=['filename', 'label'])